In [4]:
import threading
import numpy as np
import requests
import yaml

def get_data_from_endpoint():
    url = "https://api.gateio.ws/api/v4/spot/tickers"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        list_cur_date = []
        list_cur_time = []
        list_market = []
        list_symbol = []
        list_cash = []
        list_price = []

        for i in range(len(data)):
            list_market.append("gate")
            list_symbol.append(data[i]["currency_pair"])
            list_cash.append(data[i]["currency_pair"])
            list_price.append(float(data[i]["last"]))

        dict_market = {
            "MARKET": list_market,
            "SYMBOL": list_symbol,
            "CASH": list_cash,
            "PRICE": list_price,
        }
        return dict_market
    else:
        print("Failed to fetch data from the endpoint")



dict_market = get_data_from_endpoint()


In [5]:
import pandas as pd

df1 = pd.DataFrame(dict_market)

In [14]:
a = list(df1["SYMBOL"].unique())
b = []

for i in a:
    j = i.find("_")
    b.append(i[j+1:])

In [16]:
set(b)

{'BTC', 'CNYX', 'ETH', 'SGD', 'SNET', 'TRY', 'USD', 'USDC', 'USDT'}

In [1]:
import api.bybit.REST_Ticker as bybit
import api.gate_io.API_Gate_io as gate_io
import api.htx.API_HTX as htx
import api.kukoin.API_Kukoin as kukoin
import threading
from queue import Queue
import pandas as pd
import datetime


cur_date = datetime.date.today()
cur_time = datetime.datetime.now().time().strftime("%H:%M")

q1 = Queue()
# q2 = Queue()
q3 = Queue()
q4 = Queue()    

# init threads
t1 = threading.Thread(target=bybit.get_data_from_endpoint, args=[q1, cur_date, cur_time])
# t2 = threading.Thread(target=gate_io.get_data_from_endpoint, args=[q2, cur_date, cur_time])
t3 = threading.Thread(target=htx.get_data_from_endpoint, args=[q3, cur_date, cur_time])
t4 = threading.Thread(target=kukoin.get_data_from_endpoint, args=[q4, cur_date, cur_time])

# start threads
t1.start()
# t2.start()
t3.start()
t4.start()

# join threads to the main thread
t1.join()
# t2.join()
t3.join()
t4.join()

q1 = q1.get()
# q2 = q2.get()
q3 = q3.get()
q4 = q4.get()

df1 = pd.DataFrame(q1)
df3 = pd.DataFrame(q3)
df4 = pd.DataFrame(q4)

frames = [df1, df3, df4]
df = pd.concat(frames)

In [40]:
df_r = df.assign(COUNT=df.groupby(by=['SYMBOL', 'CASH'])['PRICE'].transform('count'))
df_r = df_r[df_r.COUNT > 1]
df_r = df_r.assign(MIN=df_r.groupby(by=['SYMBOL', 'CASH'])['PRICE'].transform('min'))
df_r = df_r.assign(MAX=df_r.groupby(by=['SYMBOL', 'CASH'])['PRICE'].transform('max'))

In [41]:
df_r['DIFFER'] = df_r.apply(lambda row: row.MAX - row.MIN, axis = 1)
df_r = df_r.sort_values(by=['DIFFER', 'PRICE'], ignore_index=True, ascending=False)

In [42]:
df_r = df_r.iloc[:12]

In [43]:
df_r = df_r[['DATA', 'TIME', 'MARKET', 'SYMBOL', 'CASH', 'PRICE']]

In [1]:
df_r

NameError: name 'df_r' is not defined